In [57]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import yfinance as yf
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, mean_absolute_percentage_error
from statsmodels.tsa.arima.model import ARIMA
from pmdarima import auto_arima

In [58]:
# All settings are at the top for easy changes
CONFIG = {
    "symbols": ['AMZN', '^GSPC', '^VIX', 'AAPL'],
    "feature_cols": ['GSPC', 'VIX', 'AAPL'], 
    "target_col": 'AMZN',
    "start_date": '2018-01-01',
    "end_date": '2024-12-31',
    "train_split_ratio": 0.8,
    "refit_interval": 20
}

In [59]:
print("Fetching data...")
data = yf.download(CONFIG['symbols'], start=CONFIG['start_date'], end=CONFIG['end_date'])['Close']
data = data.ffill().dropna()
data.columns = [col.replace('^', '') for col in CONFIG['symbols']]

# Immediately inspect the data
data.head()

/tmp/ipykernel_228918/1102727090.py:2: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(CONFIG['symbols'], start=CONFIG['start_date'], end=CONFIG['end_date'])['Close']
[*********************100%***********************]  4 of 4 completed

Fetching data...


,AMZN,GSPC,VIX,AAPL
Date,,,,
2018-01-02,40.426823,59.450500,2695.810059,9.77
2018-01-03,40.419785,60.209999,2713.060059,9.15
2018-01-04,40.607548,60.479500,2723.989990,9.22
2018-01-05,41.069862,61.457001,2743.149902,9.22
2018-01-08,40.917324,62.343498,2747.709961,9.52


In [60]:
print("Preparing data...")
train_size = int(len(data) * CONFIG['train_split_ratio'])
train_df, test_df = data.iloc[:train_size], data.iloc[train_size:]

y_train_log = np.log(train_df[CONFIG['target_col']])
y_test = test_df[CONFIG['target_col']]

scaler = StandardScaler()
X_train = pd.DataFrame(scaler.fit_transform(train_df[CONFIG['feature_cols']]),
                       index=train_df.index, columns=CONFIG['feature_cols'])
X_test = pd.DataFrame(scaler.transform(test_df[CONFIG['feature_cols']]),
                      index=test_df.index, columns=CONFIG['feature_cols'])

Preparing data...


In [61]:
# This cell can be re-run independently if you want to experiment with model parameters
print("Finding best ARIMA order...")
model_auto = auto_arima(y_train_log,
                        exogenous=X_train,
                        seasonal=False,
                        stationary=True,
                        trace=True,
                        stepwise=True,
                        suppress_warnings=True,
                        error_action='ignore')

# The summary is printed directly in the notebook output
best_order = model_auto.order
print(f"\nBest Order: {best_order}")
model_auto.summary()

Finding best ARIMA order...
Performing stepwise search to minimize aic


/home/michael/AI-stock-prediction-algos/.venv/lib64/python3.12/site-packages/sklearn/utils/deprecation.py:132: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/home/michael/AI-stock-prediction-algos/.venv/lib64/python3.12/site-packages/sklearn/utils/deprecation.py:132: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/home/michael/AI-stock-prediction-algos/.venv/lib64/python3.12/site-packages/sklearn/utils/deprecation.py:132: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


 ARIMA(2,0,2)(0,0,0)[0] intercept   : AIC=-6948.298, Time=0.14 sec
 ARIMA(0,0,0)(0,0,0)[0] intercept   : AIC=2271.263, Time=0.06 sec


/home/michael/AI-stock-prediction-algos/.venv/lib64/python3.12/site-packages/sklearn/utils/deprecation.py:132: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


 ARIMA(1,0,0)(0,0,0)[0] intercept   : AIC=inf, Time=0.04 sec
 ARIMA(0,0,1)(0,0,0)[0] intercept   : AIC=391.423, Time=0.10 sec


...

/home/michael/AI-stock-prediction-algos/.venv/lib64/python3.12/site-packages/sklearn/utils/deprecation.py:132: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


 ARIMA(2,0,0)(0,0,0)[0] intercept   : AIC=inf, Time=0.35 sec


/home/michael/AI-stock-prediction-algos/.venv/lib64/python3.12/site-packages/sklearn/utils/deprecation.py:132: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


 ARIMA(1,0,1)(0,0,0)[0]             : AIC=-6954.714, Time=0.41 sec

Best model:  ARIMA(1,0,1)(0,0,0)[0] intercept
Total fit time: 2.575 seconds

Best Order: (1, 0, 1)


<class 'statsmodels.iolib.summary.Summary'>
"""
                               SARIMAX Results                                
==============================================================================
Dep. Variable:                      y   No. Observations:                 1408
Model:               SARIMAX(1, 0, 1)   Log Likelihood                3481.872
Date:                Wed, 09 Jul 2025   AIC                          -6955.744
Time:                        17:04:36   BIC                          -6934.744
Sample:                             0   HQIC                         -6947.896
                               - 1408                                         
Covariance Type:                  opg                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
intercept      0.0040      0.004      1.059      0.289      -0.003       0.011
ar.L1          0.9991      0.001   1130.050      0.000       0.997       1.001
ma.L1         -0.0990      0.018     -5.430      0.000      -0.135      -0.063
sigma2         0.0004   9.41e-06     44.026      0.000       0.000       0.000
===================================================================================
Ljung-Box (L1) (Q):                   0.05   Jarque-Bera (JB):              1124.74
Prob(Q):                              0.83   Prob(JB):                         0.00
Heteroskedasticity (H):               1.14   Skew:                            -0.29
Prob(H) (two-sided):                  0.16   Kurtosis:                         7.34
===================================================================================

Warnings:
[1] Covariance matrix calculated using the outer product of gradients (complex-step).
"""

In [ ]:
print(f"Running rolling forecast (refitting every {CONFIG['refit_interval']} days)...")
history_log = list(y_train_log)
exog_history = X_train.copy()
predictions = []
model_fit = None

for i in range(len(y_test)):
    try:
        # Refit the model periodically
        if i % CONFIG['refit_interval'] == 0 or model_fit is None:
            print(f"Refitting model at step {i}...")
            model = ARIMA(history_log, exog=exog_history, order=best_order)
            
            # THE FIX IS HERE: Use method_kwargs to specify the optimizer
            model_fit = model.fit(method_kwargs={'optimizer': 'nm'})

        # Forecast the next step
        next_exog = X_test.iloc[i:i+1]
        forecast_log = model_fit.forecast(steps=1, exog=next_exog)
        prediction = np.exp(forecast_log.iloc[0])
        predictions.append(prediction)

        # Update history for the next iteration
        history_log.append(np.log(y_test.iloc[i]))
        exog_history = pd.concat([exog_history, next_exog])

    except Exception:
        print(f"Forecast failed at step {i}. Appending NaN.")
        # traceback.print_exc() # You can uncomment this for detailed debugging
        predictions.append(np.nan)

final_predictions = pd.Series(predictions, index=y_test.index).dropna()

Running rolling forecast (refitting every 20 days)...
Refitting model at step 0...


/home/michael/AI-stock-prediction-algos/.venv/lib64/python3.12/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/home/michael/AI-stock-prediction-algos/.venv/lib64/python3.12/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/home/michael/AI-stock-prediction-algos/.venv/lib64/python3.12/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)


ValueError: "nm" is not a valid estimator.

In [ ]:
aligned_actuals = y_test.loc[final_predictions.index]
rmse = np.sqrt(mean_squared_error(aligned_actuals, final_predictions))
mae = mean_absolute_error(aligned_actuals, final_predictions)
mape = mean_absolute_percentage_error(aligned_actuals, final_predictions)

print("\n--- Rolling Forecast Performance ---")
print(f"RMSE:  {rmse:.2f}")
print(f"MAE:   {mae:.2f}")
print(f"MAPE:  {mape:.2%}")

In [ ]:
plt.figure(figsize=(14, 7))
plt.plot(aligned_actuals, label='Actual Prices', color='green')
plt.plot(final_predictions, label='Rolling Forecast', linestyle='--', color='red')
plt.title(f'{CONFIG["target_col"]} Rolling Forecast vs Actual Price')
plt.xlabel('Date')
plt.ylabel('Price (USD)')
plt.legend()
plt.grid(True)
plt.show()